# IMDb Review Webscraping
Webscrape the reviews of The Last Jedi from IMDb

In [27]:
import pandas as pd
import numpy as np

import requests
requests.__path__

import re 

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

import pickle

from datetime import datetime

from selenium.webdriver.support.ui import Select

from datetime import datetime

from selenium.webdriver.support.ui import Select


import random 

import itertools

from textblob import TextBlob, Word
import langid

In [2]:
# Load pickle file with all IMDb reviews
TLJ_Reviews=pickle.load(open("IMDb_TLJ_Reviews_v1.pickle", "rb"))

## Initialize Chrome Webdriver, login to IMDb Account, and go to The Last Jedi reviews page

In [142]:
# Create list of movies that have budget and revenue numbers
# Open IMDbPro Log in page 
driver = webdriver.Chrome('/Applications/chromedriver')
driver.get('https://www.imdb.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.imdb.com%2Fap-signin-handler&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=imdb_us&openid.mode=checkid_setup&siteState=eyJvcGVuaWQuYXNzb2NfaGFuZGxlIjoiaW1kYl91cyIsInJlZGlyZWN0VG8iOiJodHRwczovL3d3dy5pbWRiLmNvbS8_cmVmXz1sb2dpbiJ9&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&tag=imdbtag_reg-20')

# Grab password and login from file
login = open('/Users/starplatinum87/Google Drive/hisoka/imdb.txt', 'r').read()
login = login.split('\n')[0:2]

# Enter email and password on login page
driver.find_element_by_xpath('//*[@id="ap_email"]').send_keys(login[0]) # Enter your own login info
driver.find_element_by_xpath('//*[@id="ap_password"]').send_keys(login[1]) # Enter your own login info
driver.find_element_by_xpath('//*[@id="signInSubmit"]').click()

# Go to reviews page
driver.get('https://www.imdb.com/title/tt2527336/reviews?ref_=tt_ov_rt')

## Get TLJ Reviews

### Get page source

In [143]:
# Get page into BeautifulSoup object 
# page = requests.get('https://www.imdb.com/title/tt2527336/reviews?ref_=tt_ov_rt')
html = driver.page_source
soup = BeautifulSoup(html)

### Load more reviews until all reviews are present

In [145]:
# Calculate number of scrolls needed
header = soup.find_all('div', class_='header')
review_num = re.findall(r'<span>(.+)\sReviews', str(header)) 
review_num = int(review_num[0].replace(',', ''))
loads = (review_num//25) + 6 # Each Load More adds 25 reviews
loads

240

In [146]:
# Load new pages until all pages are present
for i in range(loads):
    try:
        driver.find_element_by_xpath('//*[@id="load-more-trigger"]').click()
        time.sleep(10)
    except:
        continue

### Get updated page source

In [160]:
# Get page into BeautifulSoup object 
# page = requests.get('https://www.imdb.com/title/tt2527336/reviews?ref_=tt_ov_rt')
html = driver.page_source
soup = BeautifulSoup(html)

In [161]:
imdb_reviews_raw = soup.find_all('div', {'class' : re.compile("(text show-more__control|text show-more__control clickable)")})
imdb_review_list = list(imdb_reviews_raw)

### Get review text and put into list

In [162]:
# Put all reviews text into list of reviews
TAG_RE = re.compile(r'(<[^>]+>|\n)')
def remove_tags(text):
    return TAG_RE.sub("", text)

clean_imdb_reviews = []
for i in imdb_review_list:
    review = remove_tags(str(i))
    clean_imdb_reviews.append(review)
    
clean_imdb_reviews

['SPOILER: This movie was just magical.The Force has become like Harry Potter magic, with a new spell every day or so.Rey is just magical. With practically training whatsoever she can take on Luke and Kylo and win.Snoke is magical. He appeared out of nowhere in The Force Awakens and in The Last Jedi has such fantastic Force powers (never before seen in the entire Star Wars Universe) and yet can\'t tell when his protege is scheming to slice him in two (but could fell his every throught earlier, and even manipulate him and Rey\'s thoughts and get them to intergalactically Skype each other and even send water through space).Luke is magical. He can magically project himself light years away and even bring solid things (the lucky dice) with him. Then dies magically for no apparent reason.Yoda is magical. His Force-ghost actually has more Force powers than when he was alive! Calling down lightning from the sky??? Heck, why don\'t the Jedi just do that when fighting on Tattooine or whatever? 

### Pickle reviews list

In [174]:
# with open('imdb_reviews_clean_list.pickle', 'wb') as write_file:
#     pickle.dump(clean_imdb_reviews, write_file)

### Create list of scores 

In [164]:
# Get review containers soup object
imdb_review_containers = soup.find_all('div', class_="review-container")

In [165]:
# Create a Boolean list indicating whether or not a score is present
scores_list = []
for star in list(imdb_review_containers):
    if "ipl-ratings-bar" in str(star):
        scores_list.append(True)
    else:
        scores_list.append(False)

In [166]:
# Create list of actual scores on page
imdb_scores_raw = soup.find_all('div', class_="ipl-ratings-bar") # Find all reviews with any rating
imdb_scores_raw_list = list(imdb_scores_raw)


In [167]:
# Create list of scores that were left with reviews
# A number of reviews did not have scores
found_scores = re.findall("<span>(\d+)<\/span>", str(imdb_scores_raw_list))

In [168]:
# # Create list of existing scores
# found_scores = []
# for i in imdb_scores_raw_list:
#     score = re.findall("<span>(\d)<\/span>", str(i))
#     found_scores.append(score)

In [169]:
# Create list of scores with actual scores as numbers and missing scores as NaN
true_scores = []
count = 0 # Keep track of indices of found_scores list 
for i in scores_list:
    if i == True:
        true_scores.append(found_scores[count])
        count+=1
    else:
        true_scores.append(np.nan)

### Combine scores and reviews into DataFrame

In [170]:
# Create series from each list
review_series = pd.Series(clean_imdb_reviews)
scores_series = pd.Series(true_scores)

In [171]:
TLJ_Reviews = pd.DataFrame({'Reviews': review_series, 'Scores': scores_series})

In [172]:
TLJ_Reviews

,Reviews,Scores
0,SPOILER: This movie was just magical.The Force...,1
1,"This didn't feel like Star Wars. Now, I know p...",3
2,"I didn't hate TLJ, but even if I completely ig...",NaN
3,"No sense of danger, no tension, no interest an...",NaN
4,I feel America is finished. Really. The person...,NaN
5,I had relatively high expectations of the Epis...,2
6,"They are making too many movies, I think. The ...",NaN
7,The Last Jedi is obsessed with the fallacious ...,1
8,That is my prediction. IMDB will shut down use...,NaN
9,"It sucked.It was boring, it was contrived, it ...",NaN


## SUCCESS!!! ^^^
Our prototype works and we have over 5700 reviews and scores!

## Remove Non-English Reviews
Attempted to remove the non-English reviews using the langid tool, but was not successful. It took too long to run and may have just not been up to the task. Ultimately they ended up not showing up prominently in the features, so it was safe to ignore them. 

In [ ]:
# # Create list of language ids for each review
# review_list = TLJ_Reviews['Reviews']

# language_list = []
# count = 0
# for review in review_list:
#     count += 1
#     try:
#         language = langid.classify(review)[0] # Use the langid library for language detection
#         language_list.append(language)
#     except:
#         print("Error at iteration", count)
#         break

In [24]:
# # Append list of languages to DataFrame
# TLJ_Reviews['Language'] = pd.Series(language_list)

In [ ]:
# # Return the rows identified as not English
# TLJ_Reviews.loc[TLJ_Reviews['Language'] != 'en']

- Unfortunately it seems to be unreliable...